<a href="https://colab.research.google.com/github/chaitali03/RASA_Edubot/blob/main/OverviewWorkflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Noun Keyword Extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%pip install pandas
%pip install sklearn
%pip install wrapt==1.12.1 --user
%pip install sentence-transformer

import pandas as pd
import numpy as np


ERROR: Could not find a version that satisfies the requirement sentence-transformer (from versions: none)
ERROR: No matching distribution found for sentence-transformer


In [ ]:
## Need a file with the dataset initial file with necesary title defination and summarization column
#total_df = pd.read_csv('/content/gdrive/MyDrive/NarrativeArcPE/Data/selected_groupid_1.csv')
#total_df = pd.read_csv('/content/gdrive/MyDrive/selected_groupid_1.csv')
total_df = pd.read_csv('/content/drive/MyDrive/Narrative Arc PE/Data/selected_groupid_1.csv')
# total_df = pd.read_csv('/content/gdrive/MyDrive/Narrative Arc PE/Data/GroupSegmentation.csv')
#total_df = pd.read_csv('/content/selected_groupid_1.csv')
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,The pharaoh was a god and it was important not...,42,1,"['egyptian', 'article']",['egyptian government article'],NaN,"[{'egyptian': 0.81870645}, {'egyptian governme..."
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Many pharaohs did this by making war and conqu...,42,1,['egyptian pharaohs'],['egyptian pharaoh'],NaN,"[{'pharaohs': 0.92357665}, {'egyptian pharaohs..."
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,The Pharaoh was a vital part of the the Egyp...,42,1,['egypt'],['ancient egypt government'],NaN,"[{'egypt': 0.82266253}, {'ancient egypt': 0.92..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,"Red coralline alga, Kelp Forest, Plants & ...",54,1,['kelp crossword'],['kelp crossword'],NaN,"[{'crossword': 0.9012954}, {'kelp crossword': ..."
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,"It used to be used for alginate extraction, bu...",54,1,['brown seaweeds'],['brown seaweed'],NaN,"[{'seaweeds': 0.7327006}, {'brown seaweeds': 1..."
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Seaweed.ie :: Rhodophyta (Red Algae) The S...,54,1,['seaweed'],['red seaweed'],NaN,"[{'red': 0.68990666}, {'red seaweeds': 1.00000..."
6,22,207b270a-1b64-4a46-ae86-7a557f3031f2,4,e33c68c8-e30d-4f53-b96b-c6dace40a75c,Green Seaweeds,Read through this webpage to learn all about g...,f,resource,webpage_resource,http://www.seaweed.ie/algae/chlorophyta.php,Science,Green algae are thought to have the progenitor...,54,1,['seaweed'],['green seaweed'],NaN,"[{'seaweeds': 0.7568321}, {'green seaweeds': 1..."
7,25,207b270a-1b64-4a46-ae86-7a557f3031f2,13,36f7f61f-8904-4ad9-aeed-e91e096dbdf2,Red Coralline Alga,Read through this fact sheet to learn about Re...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,"Red coralline alga, Kelp Forest, Plants & ...",54,1,['coralline alga'],['red coralline alga'],NaN,"[{'red': 0.7336329}, {'red coralline': 0.95558..."
8,21,207b270a-1b64-4a46-ae86-7a557f3031f2,5,bfd5d67b-a44a-401d-807f-c37b3139f40e,Dead Man Fingers,Read through this fact sheet to learn about De...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,"Dead man's fingers, Rocky Shores, Plants & ...",54,1,"['dead', 'finger']",['dead man finger'],NaN,"[{'dead': 0.8688209}, {'dead man': 0.9459071}]"
9,27,207b270a-1b64-4a46-ae86-7a557f3031f2,8,b1bdd2f7-cc36-47ee-aa73-fb97c1bd1161,Bull Kelp,Read through this fact sheet to learn about Bu...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,"From the holdfast, a flexible stem (stipe) ext...",54,1,['bull kelp'],['bull kelp'],NaN,"[{'bull': 0.8

In [ ]:
#Text Preprocessing for title and summarization

def spaces(sample_str):
    new_str = ''
    cnt = 0
    # print(sample_str)
    if(pd.isnull(sample_str)):
      return new_str
    # print(sample_str)
    for i,char in enumerate(sample_str,0):
        # If charechter is upper and previous charechter is empty space or lower
        if(char.isupper() and (sample_str[i-1] == " " or sample_str[i-1].islower())):
#             print(i,len(new_str),sample_str[len(new_str):i],sample_str[i])
            new_str = new_str + sample_str[len(new_str)-cnt:i] + " " + sample_str[i]
            cnt += 1
    new_str = new_str + sample_str[len(new_str)-cnt:len(sample_str)]
    return new_str

def not_found(sample_str):
    new_str = sample_str
    if(pd.isnull(sample_str)):
        return new_str
#     print(sample_str)
    if('not found' in sample_str):
        new_str = np.nan
        
    return new_str


# total_df['Summarization'] = total_df['Summarization'].mask(pd.isnull, total_df['description'])

# total_df['Summarization'] = total_df['Summarization'].apply(lambda x: spaces(x))

try:
  total_df['Summarization'] = total_df['description'].mask(pd.isnull, "")
  total_df['Summarization'] = total_df['Summarization'].mask(pd.isnull, total_df['description'])

  total_df['Summarization'] = total_df['Summarization'].apply(lambda x: spaces(x))

  total_df['Summarization'] = total_df['Summarization'].apply(lambda x: not_found(x))
  # total_df[total_df['Summarization'].isna()]
except:
  print("impossible")

In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],NaN,"[{'egyptian': 0.81870645}, {'egyptian governme..."
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],NaN,"[{'pharaohs': 0.92357665}, {'egyptian pharaohs..."
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],NaN,"[{'egypt': 0.82266253}, {'ancient egypt': 0.92..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],NaN,"[{'crossword': 0.9012954}, {'kelp crossword': ..."
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],NaN,"[{'seaweeds': 0.7327006}, {'brown seaweeds': 1..."
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaweed'],NaN,"[{'red': 0.68990666}, {'red seaweeds': 1.00000..."
6,22,207b270a-1b64-4a46-ae86-7a557f3031f2,4,e33c68c8-e30d-4f53-b96b-c6dace40a75c,Green Seaweeds,Read through this webpage to learn all about g...,f,resource,webpage_resource,http://www.seaweed.ie/algae/chlorophyta.php,Science,Read through this webpage to learn all about g...,54,1,['seaweed'],['green seaweed'],NaN,"[{'seaweeds': 0.7568321}, {'green seaweeds': 1..."
7,25,207b270a-1b64-4a46-ae86-7a557f3031f2,13,36f7f61f-8904-4ad9-aeed-e91e096dbdf2,Red Coralline Alga,Read through this fact sheet to learn about Re...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about R...,54,1,['coralline alga'],['red coralline alga'],NaN,"[{'red': 0.7336329}, {'red coralline': 0.95558..."
8,21,207b270a-1b64-4a46-ae86-7a557f3031f2,5,bfd5d67b-a44a-401d-807f-c37b3139f40e,Dead Man Fingers,Read through this fact sheet to learn about De...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about D...,54,1,"['dead', 'finger']",['dead man finger'],NaN,"[{'dead': 0.8688209}, {'dead man': 0.9459071}]"
9,27,207b270a-1b64-4a46-ae86-7a557f3031f2,8,b1bdd2f7-cc36-47ee-aa73-fb97c1bd1161,Bull Kelp,Read through this fact sheet to learn about Bu...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about B...,54,1,['bull kelp'],['bull kelp'],NaN,"[{'bull': 0.85211664}, {'bu

In [ ]:
# PREPROCESSING

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
stop_words = set(stopwords.words('english')) 
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
import re , unicodedata , string

def remove_non_ascii(sample_str):
        """Remove non-ASCII characters from a sample string [sample_str]"""
        words = word_tokenize(sample_str)
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return ' '.join(new_words)

def remove_punctuation(sample_str):
    """Remove punctuation from a sample string"""
    words = word_tokenize(sample_str)
    new_words = []
    for word in words:
      new_word = re.sub(r'[^\w\s]', '', word)
      if new_word != '':
        new_words.append(new_word)
    return ' '.join(new_words)

stopword_list = set(stopwords.words('english'))
stopword_list.update(['what','a','an','the','diagram','reading','ppt','interactive','cw','quiz','hw','resource','text','choose','wwf']) 

def remove_stopwords(sample_str):
      """Remove stop words from a sample string"""
      words = word_tokenize(sample_str)
      new_words = []
      for word in words:
          if word.lower() not in stopword_list:
              new_words.append(word)
      return ' '.join(new_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Removing all NaN Titles
total_df = total_df[total_df['title'].notna()]


In [ ]:
# Removing remove_puntutations and removing all Stopwords
total_df['title'] = total_df['title'].apply(remove_punctuation).apply(remove_stopwords)
# total_df['Summarization'] = total_df['Summarization'].apply(remove_non_ascii).apply(remove_punctuation).apply(remove_stopwords)


In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],NaN,"[{'egyptian': 0.81870645}, {'egyptian governme..."
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],NaN,"[{'pharaohs': 0.92357665}, {'egyptian pharaohs..."
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],NaN,"[{'egypt': 0.82266253}, {'ancient egypt': 0.92..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],NaN,"[{'crossword': 0.9012954}, {'kelp crossword': ..."
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],NaN,"[{'seaweeds': 0.7327006}, {'brown seaweeds': 1..."
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaweed'],NaN,"[{'red': 0.68990666}, {'red seaweeds': 1.00000..."
6,22,207b270a-1b64-4a46-ae86-7a557f3031f2,4,e33c68c8-e30d-4f53-b96b-c6dace40a75c,Green Seaweeds,Read through this webpage to learn all about g...,f,resource,webpage_resource,http://www.seaweed.ie/algae/chlorophyta.php,Science,Read through this webpage to learn all about g...,54,1,['seaweed'],['green seaweed'],NaN,"[{'seaweeds': 0.7568321}, {'green seaweeds': 1..."
7,25,207b270a-1b64-4a46-ae86-7a557f3031f2,13,36f7f61f-8904-4ad9-aeed-e91e096dbdf2,Red Coralline Alga,Read through this fact sheet to learn about Re...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about R...,54,1,['coralline alga'],['red coralline alga'],NaN,"[{'red': 0.7336329}, {'red coralline': 0.95558..."
8,21,207b270a-1b64-4a46-ae86-7a557f3031f2,5,bfd5d67b-a44a-401d-807f-c37b3139f40e,Dead Man Fingers,Read through this fact sheet to learn about De...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about D...,54,1,"['dead', 'finger']",['dead man finger'],NaN,"[{'dead': 0.8688209}, {'dead man': 0.9459071}]"
9,27,207b270a-1b64-4a46-ae86-7a557f3031f2,8,b1bdd2f7-cc36-47ee-aa73-fb97c1bd1161,Bull Kelp,Read through this fact sheet to learn about Bu...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about B...,54,1,['bull kelp'],['bull kelp'],NaN,"[{'bull': 0.85211664}, {'bu

In [ ]:
# #Noun Keyword Extraction using TextBlob on brown (nltk data)

from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

lemmatizer = nltk.WordNetLemmatizer()


def get_word_postag(word):
    if pos_tag([word])[0][1].startswith('J'):
        return wordnet.ADJ
    if pos_tag([word])[0][1].startswith('V'):
        return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN
    
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    postag = get_word_postag(word)
    word = lemmatizer.lemmatize(word,postag)
    return word

#Extract noun
cols = ['Summarization','description']

for col in cols:
    total_df['Noun_keywords_' + col] = np.nan
    for i in total_df.index:
      try:
        # print(total_df[col][i],end = "-> ")
        blob = TextBlob(total_df[col][i])  
        print(total_df["title"][i],end = "-> ")
        potential_searches = []
        if (total_df[col][i] != ""):
          for nouns in blob.noun_phrases:
            # if(nouns not in englishWords):
              # print(nouns)
              potential_searches.append(normalise(nouns))
          print(potential_searches)
          total_df['Noun_keywords_' + col][i] = potential_searches
        else:
          # potential_searches = []
          total_df['Noun_keywords_' + col][i] = potential_searches
      except:
        continue

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Egyptian Government Article-> ['article', 'important figures', 'egyptian']
Egyptian Pharaohs-> ['reading', 'egyptian pharaohs']
Ancient Egypt Government-> ['egypt', 'pharaoh']
Kelp Crossword-> ['kelp forest knowledge']
Brown Seaweeds-> ['read', 'brown seaweeds']
Red Seaweeds-> ['read', 'red seaweeds']
Green Seaweeds-> ['read', 'green seaweeds']
Red Coralline Alga-> ['read', 'fact sheet', 'coralline alga', 'be', 'cool facts']
Dead Man Fingers-> ['read', 'fact sheet', 'dead', "man 's", 'finger', 'be', 'cool

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['online quiz']
Bohr Atom-> ["animation reenacts electron 's 'jump", 'precise energy levels', 'hydrogen atom']
Atoms-> ['multiple choice quiz', 'click', 'continue']
Atom-> ['wikipedia', 'atom', 'educational reference material resource', 'social sciences']
Chromosome-> ['chromosome', 'dna', 'read']
Picture Chromosomes-> ['examine', 'actual chromosomes', 'remember', 'really', 'powerful microscope']
Chromosome Structure-> ['great way', 'long strings', 'genes ... ..']
Gene-> ['wonder', 'gene relate', 'dna']
Phases Moon-> ['textbook reading', 'earth']
Activity Lunar Phases-> ['interactive resource lets', 'moon cycles']
Reviewing Phases Moon-> ['quizlet']
Tidal Energy works-> ['short article', 'tidal energy', 'different ways']
Calculating Force Gravity-> ['ck-12']
Animation Gravity Tides-> ['animation']
Gravity-> ['an', 'introductory lesson', 'ck-12', 'explore', 'gravity', 'earth', "'s gravity pulls", 'earth']
Ancient Mesopotamia Religion-> ['article', 'mesopotamia']
Ancient Egypt Legal Syst

In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords,Noun_keywords_Summarization,Noun_keywords_description
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],NaN,"[{'egyptian': 0.81870645}, {'egyptian governme...","[article, important figures, egyptian]","[article, important figures, egyptian]"
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],NaN,"[{'pharaohs': 0.92357665}, {'egyptian pharaohs...","[reading, egyptian pharaohs]","[reading, egyptian pharaohs]"
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],NaN,"[{'egypt': 0.82266253}, {'ancient egypt': 0.92...","[egypt, pharaoh]","[egypt, pharaoh]"
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],NaN,"[{'crossword': 0.9012954}, {'kelp crossword': ...",[kelp forest knowledge],[kelp forest knowledge]
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],NaN,"[{'seaweeds': 0.7327006}, {'brown seaweeds': 1...","[read, brown seaweeds]","[read, brown seaweeds]"
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaweed'],NaN,"[{'red': 0.68990666}, {'red seaweeds': 1.00000...","[read, red seaweeds]","[read, red seaweeds]"
6,22,207b270a-1b64-4a46-ae86-7a557f3031f2,4,e33c68c8-e30d-4f53-b96b-c6dace40a75c,Green Seaweeds,Read through this webpage to learn all about g...,f,resource,webpage_resource,http://www.seaweed.ie/algae/chlorophyta.php,Science,Read through this webpage to learn all about g...,54,1,['seaweed'],['green seaweed'],NaN,"[{'seaweeds': 0.7568321}, {'green seaweeds': 1...","[read, green seaweeds]","[read, green seaweeds]"
7,25,207b270a-1b64-4a46-ae86-7a557f3031f2,13,36f7f61f-8904-4ad9-aeed-e91e096dbdf2,Red Coralline Alga,Read through this fact sheet to learn about Re...,f,resource,webpage_resource,https://www.montereybayaquarium.org/animal-gui...,Science,Read through this fact sheet to learn about R...,54,1,['coralline alga'],['red coralline alga'],NaN,"[{'red': 0.7336329}, {'red coralline': 0.95558...","[read, fact sheet, coralline alga, be, cool fa...","[read, fact sheet, coralline alga, be, cool fa..."
8,21,207b270a-1b64-4a46-ae86-7a557f3031f2,5,bfd5d67b-a44a-401d-807f-c37b3139f40e,Dead Man Fingers,Read through this fact sheet to learn about De...,f,resource,webpage_resource,https://www.m

In [ ]:
#Nouns Intersection Using Sentence Segmentation

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

def get_terms(tree):    
    for leaf in leaves(tree):
        terms = [normalise(w) for w,t in leaf]
        yield terms

def leaves(tree):
  """Finds NP (nounphrase) leaf nodes of a chunk tree."""
  for subtree in tree.subtrees(filter = lambda t: t.label() =='NP'):
      yield subtree.leaves()


cols = ["Summarization","description"]

for col in cols:
    total_df['Noun_keywords_Segmentation_' + col] = np.nan
    for document in total_df.index:
      print(total_df["title"][document])
      features = []

      if (total_df[col][document] != "" and not pd.isnull(total_df[col][document])):
        # print("here")
        tokens = [nltk.word_tokenize(sent) for sent in [total_df[col][document]]]
        postag = [nltk.pos_tag(sent) for sent in tokens][0]

        
                # {<RB.?>*<VB.?>*<JJ>*<VB.?>+<VB>?} # Verbs and Verb Phrases
        grammar = r"""
            NBAR:
                {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

            NP:
                {<NBAR>}
                {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...

        """

        cp = nltk.RegexpParser(grammar)
        tree = cp.parse(postag)
        # print(tree)

        terms = get_terms(tree)

        for term in terms:
            _term = ''
            for word in term:
                _term += ' ' + word
            features.append(_term.strip())
      else:  
        features.append("[]")

      total_df['Noun_keywords_Segmentation_' + col][document] = features
      print(document,":",features)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Egyptian Government Article
0 : ['article', 'important figure', 'egyptian government']
Egyptian Pharaohs
1 : ['egyptian pharaoh']
Ancient Egypt Government
2 : ['civilization', 'rule', 'regulation', 'ancient egypt', 'government', 'single figure', 'pharaoh', 'people', 'living god']
Kelp Crossword
3 : ['kelp', 'knowledge', 'crossword puzzle']
Brown Seaweeds
4 : ['webpage', 'brown seaweed', 'phaeophyta']
Red Seaweeds
5 : ['webpage', 'red seaweed', 'rhodophyta']
Green Seaweeds
6 : ['webpage', 'green seaweed', 'chlorophyta']
Red Coralline Alga
7 : ['fact sheet', 'red coralline alga', '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)



Animation Gravity Tides
27 : ['animation', 'explanation', 'tide']
Gravity
28 : ['an introductory lesson', 'ck-12', 'gravity', 'law', 'gravity', 'earth', 'gravity', 'object', 'earth', 'nothing', 'object']
Ancient Mesopotamia Religion
29 : ['article', 'religion', 'mesopotamia']
Ancient Egypt Legal Systems
30 : ['reference article', 'ancient egyptian legal system']
Sumerian Government
31 : ['article', 'government', 'sumer', 'mesopotamia']
Fossils
32 : ['go', 'slide', 'power point presentation', 'note', 'google doc']
5 Embryo Development
33 : ['resource', 'embryo development', 'organism', 'early stage', 'growth', 'recognizable organism', 'human', 'month', 'pregnancy']
4 Homologous Structures
34 : ['resource', 'homologous structure', 'homologous structure- a body part', 'structure', 'different organism', 'performs different function']
1 Whale Fossils
35 : ['gradual evolution', 'whale fossil']
Animation Homologous Structures
36 : ['engage', 'analogous structure']
Red Seaweeds
37 : ['webpage

In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords,Noun_keywords_Summarization,Noun_keywords_description,Noun_keywords_Segmentation_Summarization,Noun_keywords_Segmentation_description
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],NaN,"[{'egyptian': 0.81870645}, {'egyptian governme...","[article, important figures, egyptian]","[article, important figures, egyptian]","[article, important figure, egyptian government]","[article, important figure, egyptian government]"
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],NaN,"[{'pharaohs': 0.92357665}, {'egyptian pharaohs...","[reading, egyptian pharaohs]","[reading, egyptian pharaohs]",[egyptian pharaoh],[egyptian pharaoh]
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],NaN,"[{'egypt': 0.82266253}, {'ancient egypt': 0.92...","[egypt, pharaoh]","[egypt, pharaoh]","[civilization, rule, regulation, ancient egypt...","[civilization, rule, regulation, ancient egypt..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],NaN,"[{'crossword': 0.9012954}, {'kelp crossword': ...",[kelp forest knowledge],[kelp forest knowledge],"[kelp, knowledge, crossword puzzle]","[kelp, knowledge, crossword puzzle]"
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],NaN,"[{'seaweeds': 0.7327006}, {'brown seaweeds': 1...","[read, brown seaweeds]","[read, brown seaweeds]","[webpage, brown seaweed, phaeophyta]","[webpage, brown seaweed, phaeophyta]"
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaweed'],NaN,"[{'red': 0.68990666}, {'red seaweeds': 1.00000...","[read, red seaweeds]","[read, red seaweeds]","[webpage, red seaweed, rhodophyta]","[webpage, red seaweed, rhodophyta]"
6,22,207b270a-1b64-4a46-ae86-7a557f3031f2,4,e33c68c8-e30d-4f53-b96b-c6dace40a75c,Green Seaweeds,Read through this webpage to learn all about g...,f,resource,webpage_resource,http://www.seaweed.ie/algae/chlorophyta.php,Science,Read through this webpage to learn all about g...,54,1,['seaweed'],['green seaweed'],NaN,"[{'seaweeds': 0.7568321}, {'green seaweeds': 1...","[read, green seaweeds]","[read, green seaweeds]","[webpage, green seaweed, chlorophyta]","[webpage, green seaweed, chlorophyta]"
7,25,207b270a-1b64-4a46-ae86-7a557f3031f2,13,36f7f

In [ ]:
# Removing Keywords of length < 3 or length > 15

def remove_unnecessary(l):
    new_l = []
    # print(type(l),l)

    if(type(l) == float):
      return new_l
    for keyword in l:
        if(len(keyword) > 3 and len(keyword) < 25):
            new_l.append(keyword)
    return new_l

total_df['Noun_keywords_Segmentation_Summarization'] = total_df['Noun_keywords_Segmentation_Summarization'].apply(lambda x: remove_unnecessary(x))
total_df['Noun_keywords_Segmentation_description'] = total_df['Noun_keywords_Segmentation_description'].apply(lambda x: remove_unnecessary(x))
total_df['Noun_keywords_description'] = total_df['Noun_keywords_description'].apply(lambda x: remove_unnecessary(x))
total_df['Noun_keywords_Summarization'] = total_df['Noun_keywords_Summarization'].apply(lambda x: remove_unnecessary(x))



In [ ]:
import ast
import itertools

total_df['All_Keywords'] = ''
with_ast_cols = []
without_ast_cols = ['Noun_keywords_Segmentation_Summarization','Noun_keywords_Segmentation_description']

all_keywords = []
cnt = 0
for i in total_df.index:
  all_keywords.append([])
  for col in with_ast_cols:
   all_keywords[cnt].extend(list(itertools.chain(*list(map(lambda x: x.split(),ast.literal_eval(total_df[col][i]))))))
  #  print(all_keywords[len(all_keywords) - 10:len(all_keywords)])
  
  for col in without_ast_cols: 
   all_keywords[cnt].extend(list(itertools.chain(*list(map(lambda x: x.split(),total_df[col][i])))))
   print(all_keywords[len(all_keywords) - 10:len(all_keywords)])
   
  cnt += 1
  # total_df['All_Keywords'][i] = list(set(total_df['EmbbedRank'][i]).union(set(total_df['keyPhrases_des'][i])).union(set(total_df['Noun_keywords'][i])))
total_df['All_Keywords'] = all_keywords

[['article', 'important', 'figure', 'egyptian', 'government']]
[['article', 'important', 'figure', 'egyptian', 'government', 'article', 'important', 'figure', 'egyptian', 'government']]
[['article', 'important', 'figure', 'egyptian', 'government', 'article', 'important', 'figure', 'egyptian', 'government'], ['egyptian', 'pharaoh']]
[['article', 'important', 'figure', 'egyptian', 'government', 'article', 'important', 'figure', 'egyptian', 'government'], ['egyptian', 'pharaoh', 'egyptian', 'pharaoh']]
[['article', 'important', 'figure', 'egyptian', 'government', 'article', 'important', 'figure', 'egyptian', 'government'], ['egyptian', 'pharaoh', 'egyptian', 'pharaoh'], ['civilization', 'rule', 'regulation', 'ancient', 'egypt', 'government', 'single', 'figure', 'pharaoh', 'people', 'living', 'god']]
[['article', 'important', 'figure', 'egyptian', 'government', 'article', 'important', 'figure', 'egyptian', 'government'], ['egyptian', 'pharaoh', 'egyptian', 'pharaoh'], ['civilization', 'rul

In [ ]:
def GetNGrams(x,n_gram_range):
  x = ' '.join(x)
  if(x != ""):
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([x])
    candidates = count.get_feature_names()
  else:
    candidates = x.split()
  return candidates
total_df['NGramGeneratedAllKeywords'] = list(map(lambda x: GetNGrams(x,(1,4)),all_keywords))

In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords,Noun_keywords_Summarization,Noun_keywords_description,Noun_keywords_Segmentation_Summarization,Noun_keywords_Segmentation_description,NGramGeneratedAllKeywords
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],"[article, important, figure, egyptian, governm...","[{'egyptian': 0.81870645}, {'egyptian governme...","[article, important figures, egyptian]","[article, important figures, egyptian]","[article, important figure, egyptian government]","[article, important figure, egyptian government]","[article, article important, article important..."
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],"[egyptian, pharaoh, egyptian, pharaoh]","[{'pharaohs': 0.92357665}, {'egyptian pharaohs...","[reading, egyptian pharaohs]","[reading, egyptian pharaohs]",[egyptian pharaoh],[egyptian pharaoh],"[egyptian, egyptian pharaoh, egyptian pharaoh ..."
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],"[civilization, rule, regulation, ancient, egyp...","[{'egypt': 0.82266253}, {'ancient egypt': 0.92...","[egypt, pharaoh]","[egypt, pharaoh]","[civilization, rule, regulation, ancient egypt...","[civilization, rule, regulation, ancient egypt...","[ancient, ancient egypt, ancient egypt governm..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],"[kelp, knowledge, crossword, puzzle, kelp, kno...","[{'crossword': 0.9012954}, {'kelp crossword': ...",[kelp forest knowledge],[kelp forest knowledge],"[kelp, knowledge, crossword puzzle]","[kelp, knowledge, crossword puzzle]","[crossword, crossword puzzle, crossword puzzle..."
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],"[webpage, brown, seaweed, phaeophyta, webpage,...","[{'seaweeds': 0.7327006}, {'brown seaweeds': 1...","[read, brown seaweeds]","[read, brown seaweeds]","[webpage, brown seaweed, phaeophyta]","[webpage, brown seaweed, phaeophyta]","[brown, brown seaweed, brown seaweed phaeophyt..."
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaweed'],"[webpage, red, seaweed, rhodophyta, webpage, r...","[{'red': 0.68990666}, {'red seaweeds': 1.00000...","[read, red seaweeds]","[read, red seaweeds]","[webpage, red seaweed, rhodophyta]","[webpage, red seawee

# Intersection Between Keywords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Wiki Dataset For similarity scores, we  would be using the doc provided here to generate a Word2vec similarity model to get similar keywords
'''
Note  : This is not the best approach since a lot of noise data comes in here which we saw when we tested it as well
'''

 
from gensim.models import Phrases
 
from gensim.models import Phrases
 
f = open('/content/drive/MyDrive/Narrative Arc PE/Data/New_Dataset_With_Better_Keywords/preprocessedData_optionGeneration.txt')
unigram_sentences = f.readlines()
 
unigram_sentences = set(unigram_sentences)
 
unigram_sentences = list(unigram_sentences)
 
 
unigram_sentences = [sen[:len(sen) - 1] for sen in unigram_sentences]

while '' in unigram_sentences:
    unigram_sentences.remove('')
# unigram_sentences
 
unigram_sentences = [doc for doc in unigram_sentences]

split_sentences = [doc.split() for doc in unigram_sentences]
bigram_phraser = Phrases(split_sentences, min_count=1, threshold= 80)

i = 10
bigram_phraser[split_sentences][i] , split_sentences[i]

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


(['check', 'whether', 'respondents', 'paying_attention', 'instructions'],
 ['check', 'whether', 'respondents', 'paying', 'attention', 'instructions'])

In [ ]:
# bigram['egyptian government'.split()]

In [ ]:
%pip install sentence_transformers
from sentence_transformers import SentenceTransformer
 
model = SentenceTransformer('bert-base-nli-mean-tokens')

# del model

     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 2.9 MB 27.7 MB/s 
     |████████████████████████████████| 3.3 MB 50.6 MB/s 
     |████████████████████████████████| 1.2 MB 50.9 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 75.2 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=b2dbc8ec13f1c83d7c3ca549130193e3da0dc24866a3eecf6050be5cc98a91cb
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Setting up our daataset for similarity scores, could use multiple kyewords or single keywords

import ast
def fun(x):
  max = 0
  max_key = None
  for i in ast.literal_eval(x):
    if(type(i) == dict):
      if(list(i.values())[0] > max):
        max = list(i.values())[0]
        max_key = list(i.keys())[0]
    elif(type(i) == str):
      if(max_key == None):
        max_key = i
  return max_key
 
a = np.array(total_df[['Noun_keywords_Segmentation_Summarization']].values.tolist())

def all_keys(a):
  new_key = set()
  for elem in a:
    print(elem)
    for key in elem:
      new_key.add(key)
  return list(new_key)
total_df['keyword_needed'] = list(map(all_keys,a))

# print(a,"\n\n\n",np.vstack((a[:,0],a[:,1])),len(np.hstack((a[:,0],a[:,1]))))

# total_df['All_Noun_Keywords'] = np.hstack((a[:, 0], a[:, 1],a[:, 2],a[:, 3])).tolist()

['article', 'important figure', 'egyptian government']
['egyptian pharaoh']
['civilization', 'rule', 'regulation', 'ancient egypt', 'government', 'single figure', 'pharaoh', 'people', 'living god']
['kelp', 'knowledge', 'crossword puzzle']
['webpage', 'brown seaweed', 'phaeophyta']
['webpage', 'red seaweed', 'rhodophyta']
['webpage', 'green seaweed', 'chlorophyta']
['fact sheet', 'red coralline alga', 'important red seaweed', 'cool fact']
['fact sheet', 'dead man', 'finger', 'important green seaweed', 'cool fact']
['fact sheet', 'bull kelp', 'important brown seaweed', 'cool fact']
['video mr. andersen', 'atomic structure', 'periodic table']
['student', 'knowledge', 'matter', 'online quiz']
['animation', "'jump", 'energy level', 'hydrogen atom']
['atom', 'molecule', 'compound', 'answer', 'explanation', 'next question', 'quiz']
['wikipedia present atom', 'social science']
['chromosome', 'package', 'parent', 'child', 'webpage', 'chromosome', 'role', 'trait']
['examine', 'picture', 'actual

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
total_df = total_df[total_df['keyword_needed'].notna()]

In [ ]:
values = total_df.groupby('group_id')['group_id'].apply(lambda x: len(x) >= 2)
 
req_gp_ids = []
for group_id in values.index:
  if(values[group_id]):
    req_gp_ids.append(group_id)
 
# values

In [ ]:
grouped = total_df[total_df['group_id'].apply(lambda x: x in req_gp_ids)].groupby('group_id')
 
new_df = pd.DataFrame()
 
new_df['group_id'] = np.nan
new_df['keywords'] = np.nan
# new_df
all_gps = []
all_keywords = []

def all_keys(a):
  new_key = set()
  for elem in a:
    # print(elem)
    for key in elem:
      new_key.add(key)
  return list(new_key)

for name,group in grouped:
  all_gps.append(name)
  # print(type(group['keyword_needed'].values))
  all_keywords.append(group['keyword_needed'].values)

print()
  
print(len(all_gps))
new_df['group_id'] = all_gps
new_df['keywords'] = list(map(all_keys,all_keywords))


10


In [ ]:
new_df

,group_id,keywords
0,42,"[egyptian government, ancient egypt, regulatio..."
1,54,"[dead man, crossword puzzle, important brown s..."
2,214,"[wikipedia present atom, student, 'jump, compo..."
3,252,"[nucleus, role, trait, child, parent, powerful..."
4,421,"[short quizlet, moon, student, understand, mov..."
5,452,"[different way, tidal energy work, short article]"
6,556,"[gravity, animation, an introductory lesson, o..."
7,740,"[article, reference article, mesopotamia, reli..."
8,795,"[homologous structure, recognizable organism, ..."
9,2307,"[dead man, crossword puzzle, important brown s..."


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
%pip install gingerit
text = 'conservation massconservation mass'
from gingerit.gingerit import GingerIt
parser = GingerIt()

def find_lemmatized_words(words):
  # words = Rule1(word,out)
  # print(word,words)
  flag = False
  # See this library for lemmatization instead of stemming
  # words = 
  print
  for i in range(len(words)):
    words[i] = ' '.join(dict.fromkeys(words[i].split()))
    for w in words[i].split():
      if(len(w) > 20):
        continue
    print(words[i],end=":")
    words[i] = parser.parse(lemmatizer.lemmatize(words[i].lower()))['result']
    print("Lemmatized Word:", words[i],end = "<->")

  list_set = set(words)
  # convert the set to the list
  unique_list = (list(list_set))
  print()
  return unique_list

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
     |████████████████████████████████| 62 kB 959 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
new_df['keywords'] = new_df['keywords'].apply(find_lemmatized_words)

egyptian government:Lemmatized Word: Egyptian government<->ancient egypt:Lemmatized Word: ancient Egypt<->regulation:Lemmatized Word: regulation<->living god:Lemmatized Word: living God<->article:Lemmatized Word: article<->important figure:Lemmatized Word: important figure<->people:Lemmatized Word: people<->pharaoh:Lemmatized Word: Pharaoh<->egyptian pharaoh:Lemmatized Word: Egyptian Pharaoh<->single figure:Lemmatized Word: single figure<->civilization:Lemmatized Word: civilization<->rule:Lemmatized Word: rule<->government:Lemmatized Word: government<->
dead man:Lemmatized Word: dead man<->crossword puzzle:Lemmatized Word: crossword puzzle<->important brown seaweed:Lemmatized Word: important brown seaweed<->important green seaweed:Lemmatized Word: important green seaweed<->important red seaweed:Lemmatized Word: important red seaweed<->fact sheet:Lemmatized Word: fact sheet<->bull kelp:Lemmatized Word: bull kelp<->cool fact:Lemmatized Word: cool fact<->brown seaweed:Lemmatized Word: bro

In [ ]:
new_df

,group_id,keywords
0,42,"[regulation, living God, article, ancient Egyp..."
1,54,"[dead man, crossword puzzle, important brown s..."
2,214,"[student, 'jump, compound, Wikipedia present a..."
3,252,"[nucleus, role, trait, child, parent, powerful..."
4,421,"[moon, student, phase, understand, interactive..."
5,452,"[different way, tidal energy work, short article]"
6,556,"[gravity, animation, explanation, lesson, an i..."
7,740,"[article, reference article, religion, Mesopot..."
8,795,"[homologous structure, recognizable organism, ..."
9,2307,"[dead man, crossword puzzle, important brown s..."


In [ ]:
# !pip install gingerit

In [ ]:
# doc = unigram_sentences

# For faster run you can change the doc to compare the keywords for TFIDF in BERT
# use the below keywords

import pandas as pd
test_df = pd.read_csv('/content/drive/MyDrive/Narrative Arc PE/Data/New_Dataset_With_Better_Keywords/New_Dataset_Keywords_Multiple.csv')

# # Setting up our daataset for similarity scores, could use multiple kyewords or single keywords

import ast
def fun(x):
  max = 0
  max_key = None
  for i in ast.literal_eval(x):
    if(type(i) == dict):
      if(list(i.values())[0] > max):
        max = list(i.values())[0]
        max_key = list(i.keys())[0]
    elif(type(i) == str):
      if(max_key == None):
        max_key = i
  return max_key
 
test_df['keyword_needed'] = test_df['BERT_Keywords'].apply(lambda x: fun(x))

doc = test_df[test_df['keyword_needed'].notna()]['keyword_needed'].unique()
# test_df['BERT_Keywords']

In [ ]:
doc

array(['powerpoint slide', 'burning campfire', 'paper notebook', ...,
       'lake river', 'plant escape', 'make lake'], dtype=object)

In [ ]:
!pip install annoy

     |████████████████████████████████| 646 kB 30.1 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391684 sha256=5f7cfe7a4b4f3ac1aee17c52dec83406627d8158428df5a35bfebd911a3ece7f
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [ ]:
# split_sentences = [doc.split() for doc in unigram_sentences]
# split_sentences.extend(list(map(lambda x: x.split(),doc_new)))
# len(split_sentences[-1])
bigram = Phrases(split_sentences,threshold=0.001)

import numpy as np
print(len(split_sentences))
data = np.hstack([split_sentences,bigram])

162613


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.word2vec import Text8Corpus

# Using params from Word2Vec_FastText_Comparison

params = {
    'alpha': 0.05,
    'size': 100,
    'window': 5,
    'iter': 10,
    'min_count': 1,
    'sample': 1e-4,
    'sg': 1,
    'hs': 0,
    'negative': 5,
}

word_model = Word2Vec(bigram_phraser[split_sentences], **params)
wv = word_model.wv
print("Using trained model", wv.vocab)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# %pip install polyglot

In [ ]:
doc

array(['powerpoint slide', 'burning campfire', 'paper notebook', ...,
       'lake river', 'plant escape', 'make lake'], dtype=object)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import pickle
stop_words = "english"
 
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


# Vectorizer = TfidfTransformer(smooth_idf=True,use_idf=True) 
# sentence_embeddings = []
 
def setUpSentence(candidates=None,file_name = 'sen_embed.pkl'):
  sentence_embeddings = []
  for n_gram_range in [(1,3)]:
    if(type(candidates) == type(None)):
      count = TfidfVectorizer(smooth_idf=True,use_idf=True,ngram_range=n_gram_range, stop_words=stop_words).fit(doc)
      candidates = count.get_feature_names()
      # candidates = count.get_feature_names()
    sentence_embeddings.append(model.encode(candidates))
  
  with open(file_name, 'wb') as f:
    pickle.dump(sentence_embeddings, f)

  return sentence_embeddings


# doc = 
def Annoy_Nearest_words(word,word_model):
  wv = word_model.wv

  # 100 trees are being used in this example
  annoy_index = AnnoyIndexer(word_model, 100)
  # Derive the vector for the word "egyptian government" in our model
  vector = wv[word]
  # print(vector)
  # The instance of AnnoyIndexer we just created is passed
  approximate_neighbors = wv.most_similar([vector], topn=11, indexer=annoy_index)
  # Neatly print the approximate_neighbors and their corresponding cosine similarity values
  # print("Approximate Neighbors")
  # for neighbor in approximate_neighbors:
  #   print(neighbor)

  # normal_neighbors = wv.most_similar([vector], topn=11)
  # print("\nExact Neighbors")
  # for neighbor in normal_neighbors:
  #   print(neighbor)

  return approximate_neighbors

def generate_nearest_words_bert(word, candidates, sentence_embeddings, top_n = 5, load_file = False, tfidf_weight = 2):
  keywords = []
  Vectorizers = []
  #try:
  #   neighbors = Annoy_Nearest_words('_'.join(word.split()),word_model) 
  #  keys = map(lambda x: x[0],neighbors)
  #  print(keys)

  #  keywords.extend(get_similarity_score(keys,word))
  #except:
  # keywords = []
  
  # print(keywords)

  for i in range(1):  

    # top_n = len(candidates)
    word_embedding = model.encode([word])  
  
    if(not sentence_embeddings):
      sentence_embeddings = setUpSentence()
    
    distances = cosine_similarity(word_embedding,sentence_embeddings[i])
    
    count = TfidfVectorizer(smooth_idf=True,use_idf=True,ngram_range=(1,2), stop_words=stop_words).fit(doc)
    
    if(type(candidates) == type(None)):
      candidates = count.get_feature_names()
    
    Vectorizers.append(count)
    norm = np.linalg.norm(Vectorizers[i].idf_)
    new_mul = Vectorizers[i].idf_
    top_n = min(top_n,len(candidates))
    
    def test(item):
      if (list(item.keys())[0] in list(Vectorizers[i].vocabulary_.keys())):
        return new_mul[list(Vectorizers[i].vocabulary_.keys()).index(list(item.keys())[0])] * tfidf_weight
      else:
        return 1
    
    top_keywords = [{candidates[index] : distances[0][index]} for index in distances.argsort()[0][-top_n:]]
    # print(top_keywords)
    top_keywords = [{list(item.keys())[0] :  list(item.values())[0] / test(item)} for item in top_keywords if(word not in list(item.keys()))]
    out_keywords = sorted(top_keywords, key=lambda k: list(k.values())[0],reverse = True)[-top_n:]
    # print(keywords) 
    keywords.extend(out_keywords)
    # print
  print(word, ":" , keywords)
  print()
  
  # all_keywords = sorted(keywords, key=lambda k: list(k.values())[0],reverse = True) 
  return keywords

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
generate_nearest_words_bert('refraction',None,None)

refraction : [{'refraction wave': 0.05755906434973439}, {'reflection refraction': 0.054470630968186086}, {'wavelength': 0.040702667469623005}, {'wavelength wave': 0.038941826385349355}]



[{'refraction wave': 0.05755906434973439},
 {'reflection refraction': 0.054470630968186086},
 {'wavelength': 0.040702667469623005},
 {'wavelength wave': 0.038941826385349355}]

In [ ]:
# doc = unigram_sentences
# setUpSentence()

In [ ]:
# Zeroshot Gandharv
# Template Gandharv
# Get intersection within regions
 
 
def intersection_keywords(sections):
  current_words = {}
  intersected_words = {}
  for word in sections:
    for token in sections[word]:
      # print(token)
      if(list(token.keys())[0] not in list(current_words.keys())):
        current_words[list(token.keys())[0]] = 1
      else:
        if(list(token.keys())[0] not in list(intersected_words.keys())):
          intersected_words[list(token.keys())[0]] = current_words[list(token.keys())[0]] + 1
        else:
          intersected_words[list(token.keys())[0]] += 1
 
  # print(intersected_words)
  if(len(list(sections.keys())) < 2):
    intersected_words = sections[word]
 
  return intersected_words
 
def Intersection_Between(keywords,candidates=None):
  keys = keywords
  completed_words = []
  nearest_words = {}
  output_to_append = []
  cnt = 0
  
  sentence_embeddings = setUpSentence(candidates)
  
  for word in keys:
    if (word not in completed_words):
      if (type(candidates) == type(None)):
        nearest_words[word] = generate_nearest_words_bert(word,candidates,sentence_embeddings)
      completed_words.append(word)
      output_to_append = intersection_keywords(nearest_words)
      print(word,end="<->")
      print(output_to_append)
      print()
      # print(cnt)
      cnt += 1     
    else:
      continue
  # final_output.append(output_to_append)
 
# output_to_append = max(key=output_to_append, key = output_to_append.get)
  return output_to_append
 
# final_output = []
#for idx in new_df.index:
  #gp_id = new_df['group_id'][idx]
  #output_to_append = Intersection_Between(new_df['keywords'][idx])
  #final_output.append(output_to_append)
  #print(output_to_append)

In [ ]:
def get_similarity_score(keywords, word):
  word_embedding = model.encode([word])

  score = []
  for k in keywords:
    keyword_embedding = model.encode([k])
    score.append({k : cosine_similarity(word_embedding,keyword_embedding)[0][0]})
  
  score = sorted(score, key=lambda k: list(k.values())[0],reverse = True)
  return score

In [ ]:
# f = open("/content/gdrive/MyDrive/Narrative Arc PE/IntersectionLevel2_new_TFIDF_(1,2).txt","a")

def Intersection_Between_level2(keywords,candidates):
  words = {}  

  # f = open("/content/gdrive/MyDrive/Narrative Arc PE/IntersectionLevel2_new_TFIDF_(1,2).txt","a")

  
  print("X-X-X-X-X-X-X-         Running Level 1 Keywords : ",keywords, "       -X-X-X-X-X-X-X")
  cnt = 0
  for k in keywords:
    sentence_embeddings = setUpSentence(candidates[cnt])
    nearest_words = generate_nearest_words(k,candidates[cnt],sentence_embeddings)
    for d in nearest_words:
      for w in d.keys():
        if w not in words:
          words[w] = 0
        words[w] += 1
        #print(d)

  # level 2

  level2_keywords = list(words.keys())
  
  print("X-X-X-X-X-X-X-         Running Level 2 Keywords : ",level2_keywords,"        -X-X-X-X-X-X-X")
  # Uncomment and Change the doc variable here to the list of keywords you want to compare the level2_keywords with
  # to get the intersection
  # or if you have a list of candidates use them here
  
  # doc = 
  # candidates =
  
  for k in level2_keywords:
    sentence_embeddings = setUpSentence()
    nearest_words = generate_nearest_words(k,candidates,sentence_embeddings)
    for d in nearest_words:
      for w in d.keys():
        if w not in words:
          words[w] = 0
        words[w] += 1
        #print(d)

  intersection = [w for w in words.keys() if words[w] > 1]
  intersection = [(w,get_similarity_score(keywords,w)) for w in intersection]
  intersection = sorted(intersection, key = lambda x : x[1], reverse = True)

  # f.write(str(keywords) + "\n" + str(intersection) + "\n" + "-"*30 + "\n")

  # f.close()
  return intersection

In [ ]:
def index(key,value,all_data):
  for keys in all_data:
    if(list(keys.keys())[0] == key):
      all_data[all_data.index(keys)][key] += value
  return all_data 

def ChooseTopNForSingleKeyword(keyword,candidates,top_n):
  cnt = 0
  completed_words = []
  all_data = []
  sentence_embeddings = setUpSentence(candidates)
  print(sentence_embeddings)
  if(candidates != []):
    nearest_words = generate_nearest_words(keyword,candidates,sentence_embeddings)
  else:
    nearest_words = []
  for word in nearest_words[0:min(top_n,len(nearest_words))]:
    print(completed_words)
    if(list(word.keys())[0] not in completed_words):
      all_data.append(word)
    else:
      all_data = index(list(word.keys())[0],list(word.values())[0],all_data)
    completed_words.append(list(word.keys())[0])
  return all_data
  
def ChooseTopNFromAllCandidates(keywords,candidates,top_n):
  cnt = 0
  all_data = []
  completed_words = []
  for k in keywords:
    sentence_embeddings = setUpSentence(candidates[cnt])
    # 
    if(candidates[cnt] != []):
      nearest_words = generate_nearest_words(k,candidates[cnt],sentence_embeddings)
      print(nearest_words)
    else:
      nearest_words = []
    for word in nearest_words[0:min(top_n,len(nearest_words))]:
      print(completed_words)
      if(list(word.keys())[0] not in completed_words):
        all_data.append(word)
      else:
        all_data = index(list(word.keys())[0],list(word.values())[0],all_data)
      completed_words.append(list(word.keys())[0])
    print(all_data)
    cnt+=1
  return all_data

In [ ]:
a = []
b = [3,4,5]
c = [6,7]
a.extend(b)
print(a)
a = a.extend(c)
a

[3, 4, 5]


In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords,Noun_keywords_Summarization,Noun_keywords_description,Noun_keywords_Segmentation_Summarization,Noun_keywords_Segmentation_description,NGramGeneratedAllKeywords,keyword_needed
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],"[article, important, figure, egyptian, governm...","[{'egyptian': 0.81870645}, {'egyptian governme...","[article, important figures, egyptian]","[article, important figures, egyptian]","[article, important figure, egyptian government]","[article, important figure, egyptian government]","[article, article important, article important...","[egyptian government, article, important figure]"
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],"[egyptian, pharaoh, egyptian, pharaoh]","[{'pharaohs': 0.92357665}, {'egyptian pharaohs...","[reading, egyptian pharaohs]","[reading, egyptian pharaohs]",[egyptian pharaoh],[egyptian pharaoh],"[egyptian, egyptian pharaoh, egyptian pharaoh ...",[egyptian pharaoh]
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],"[civilization, rule, regulation, ancient, egyp...","[{'egypt': 0.82266253}, {'ancient egypt': 0.92...","[egypt, pharaoh]","[egypt, pharaoh]","[civilization, rule, regulation, ancient egypt...","[civilization, rule, regulation, ancient egypt...","[ancient, ancient egypt, ancient egypt governm...","[ancient egypt, regulation, living god, people..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],"[kelp, knowledge, crossword, puzzle, kelp, kno...","[{'crossword': 0.9012954}, {'kelp crossword': ...",[kelp forest knowledge],[kelp forest knowledge],"[kelp, knowledge, crossword puzzle]","[kelp, knowledge, crossword puzzle]","[crossword, crossword puzzle, crossword puzzle...","[knowledge, kelp, crossword puzzle]"
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],"[webpage, brown, seaweed, phaeophyta, webpage,...","[{'seaweeds': 0.7327006}, {'brown seaweeds': 1...","[read, brown seaweeds]","[read, brown seaweeds]","[webpage, brown seaweed, phaeophyta]","[webpage, brown seaweed, phaeophyta]","[brown, brown seaweed, brown seaweed phaeophyt...","[webpage, brown seaweed, phaeophyta]"
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaw

In [ ]:
# This it the main cell for getting the Keyword Intersection
 
keywords,candidates = total_df['title'].values,total_df['keyword_needed'].values

grouped = total_df.groupby('group_id')

all_gps = []
all_keywords = []
all_candidates = []
flattened_candidates = []
# Change These fields for Changing how Keyword Comparision Takes Place
candidate_col = "keyword_needed"
truth_col = "title"

i = 0
keywords_for_overview = []
for name,group in grouped:
  all_gps.append(name)
  # print(type(group['keyword_needed'].values))
  all_candidates.append(group[candidate_col].values)
  all_keywords.append(group[truth_col].values)
  

# print(all_candidates)

i = 0
for candidate in all_candidates:
  to_append = []
  # print(candidate)
  # print()
  for key in candidate:
    # print(key)
    to_append.extend(key)
  # print(to_append,candidate)
  # print()
  flattened_candidates.append(to_append)
  # print(flattened_candidates[i])
  # print()
  i += 1

# print(flattened_candidates)
# print(all_candidates)


cnt = 0
intersect = []
for i in range(len(all_keywords)):
  keywords = all_keywords[i]
  candidates = flattened_candidates[i]
  print(keywords,candidates)
  
#   # Note if you do Intersection_Between_Level_2 
#   # You would need a set of candidates keywords somehow extracted for it as well 
#   # or else this won't work
#   # if you see doc not defined set doc = unigram_sentences
#   # as global variable

  temp = []
  candidates = []
  for k in range(len(keywords)):
    print(k," Title : " , keywords[k])
    # print(all_candidates[i][k],keywords[k])
    if(all_candidates[i][k] != []):
      out = get_similarity_score(all_candidates[i][k],keywords[k])
      print(out)
      temp.append(list(out[0].keys())[0])
      candidates.extend(list(map(lambda x: list(x.keys())[0],out[0:min(3,len(out))])))
    else:
      continue
  keywords_for_overview.append(temp)

#   # print()
  print(candidates)
  out = Intersection_Between(candidates)
  intersect.append(out)
  print(out)
  print()
  # intersect.append(Intersection_Between(keywords,candidates))




['Egyptian Government Article' 'Egyptian Pharaohs'
 'Ancient Egypt Government'] ['egyptian government', 'article', 'important figure', 'egyptian pharaoh', 'ancient egypt', 'regulation', 'living god', 'people', 'pharaoh', 'single figure', 'civilization', 'rule', 'government']
0  Title :  Egyptian Government Article
[{'egyptian government': 0.90641826}, {'article': 0.49047285}, {'important figure': 0.13345954}]
1  Title :  Egyptian Pharaohs
[{'egyptian pharaoh': 0.9671538}]
2  Title :  Ancient Egypt Government
[{'ancient egypt': 0.8853798}, {'pharaoh': 0.642393}, {'government': 0.55972373}, {'civilization': 0.4477548}, {'rule': 0.2342156}, {'people': 0.15911102}, {'regulation': 0.15672272}, {'living god': 0.12626815}, {'single figure': 0.01047584}]
['egyptian government', 'article', 'important figure', 'egyptian pharaoh', 'ancient egypt', 'pharaoh', 'government']
egyptian government : [{'egyptian': 0.05071388756453335}, {'egypt': 0.04928426020529697}, {'ancient egypt': 0.0449654230287055

In [ ]:
total_df

,Unnamed: 0,collection_id,sequence_id,resource_id,title,description,is_deleted,content_format,content_subformat,url,subject,Summarization,group_id,new_group_id,Noun_keywords_title,Noun_keywords_Segmentation_title,All_Keywords,BERT_Keywords,Noun_keywords_Summarization,Noun_keywords_description,Noun_keywords_Segmentation_Summarization,Noun_keywords_Segmentation_description,NGramGeneratedAllKeywords,keyword_needed
0,4635,dbbd449c-1612-4e53-89b7-5c5b7898422b,4,1a64082b-9aab-47ea-b36e-81ad6d298826,Egyptian Government Article,Article on important figures in Egyptian gover...,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-governm...,Science,Article on important figures in Egyptian gove...,42,1,"['egyptian', 'article']",['egyptian government article'],"[article, important, figure, egyptian, governm...","[{'egyptian': 0.81870645}, {'egyptian governme...","[article, important figures, egyptian]","[article, important figures, egyptian]","[article, important figure, egyptian government]","[article, important figure, egyptian government]","[article, article important, article important...","[egyptian government, article, important figure]"
1,4658,dbbd449c-1612-4e53-89b7-5c5b7898422b,2,c32ed5e8-1ca6-440d-a6c4-e43ea1a48873,Egyptian Pharaohs,Reading on the Egyptian Pharaohs,f,resource,webpage_resource,http://www.historyforkids.net/egyptian-pharaoh...,Science,Reading on the Egyptian Pharaohs,42,1,['egyptian pharaohs'],['egyptian pharaoh'],"[egyptian, pharaoh, egyptian, pharaoh]","[{'pharaohs': 0.92357665}, {'egyptian pharaohs...","[reading, egyptian pharaohs]","[reading, egyptian pharaohs]",[egyptian pharaoh],[egyptian pharaoh],"[egyptian, egyptian pharaoh, egyptian pharaoh ...",[egyptian pharaoh]
2,4656,dbbd449c-1612-4e53-89b7-5c5b7898422b,3,a0412b5c-27b8-4f0c-912e-5841b3cab2c9,Ancient Egypt Government,Every civilization is built upon rules and reg...,f,resource,webpage_resource,http://www.ancient-egypt-online.com/ancient-eg...,Science,Every civilization is built upon rules and reg...,42,1,['egypt'],['ancient egypt government'],"[civilization, rule, regulation, ancient, egyp...","[{'egypt': 0.82266253}, {'ancient egypt': 0.92...","[egypt, pharaoh]","[egypt, pharaoh]","[civilization, rule, regulation, ancient egypt...","[civilization, rule, regulation, ancient egypt...","[ancient, ancient egypt, ancient egypt governm...","[ancient egypt, regulation, living god, people..."
3,26,207b270a-1b64-4a46-ae86-7a557f3031f2,10,a4d1c64b-2dc9-49ee-a74d-899e206bf46f,Kelp Crossword,Use your kelp forest knowledge to complete thi...,f,resource,text_resource,http://www.montereybayaquarium.org/-/m/pdf/edu...,Science,Use your kelp forest knowledge to complete thi...,54,1,['kelp crossword'],['kelp crossword'],"[kelp, knowledge, crossword, puzzle, kelp, kno...","[{'crossword': 0.9012954}, {'kelp crossword': ...",[kelp forest knowledge],[kelp forest knowledge],"[kelp, knowledge, crossword puzzle]","[kelp, knowledge, crossword puzzle]","[crossword, crossword puzzle, crossword puzzle...","[knowledge, kelp, crossword puzzle]"
4,23,207b270a-1b64-4a46-ae86-7a557f3031f2,7,83b32fc7-003d-4fe9-ab0e-4a9b9282be4b,Brown Seaweeds,Read through this webpage to learn all about b...,f,resource,webpage_resource,http://www.seaweed.ie/algae/phaeophyta.php,Science,Read through this webpage to learn all about b...,54,1,['brown seaweeds'],['brown seaweed'],"[webpage, brown, seaweed, phaeophyta, webpage,...","[{'seaweeds': 0.7327006}, {'brown seaweeds': 1...","[read, brown seaweeds]","[read, brown seaweeds]","[webpage, brown seaweed, phaeophyta]","[webpage, brown seaweed, phaeophyta]","[brown, brown seaweed, brown seaweed phaeophyt...","[webpage, brown seaweed, phaeophyta]"
5,24,207b270a-1b64-4a46-ae86-7a557f3031f2,12,47521df4-4523-4cd1-b926-a81ae568e3c6,Red Seaweeds,Read through this webpage to learn all about r...,f,resource,webpage_resource,http://www.seaweed.ie/algae/rhodophyta.php,Science,Read through this webpage to learn all about r...,54,1,['seaweed'],['red seaw

In [ ]:


new_df = pd.DataFrame(np.array([all_gps,intersect,keywords_for_overview]).T)
new_df = new_df.rename({0 : 'gp_id',1 : 'intersect_keywords',2 : 'all_keywords'},axis = 1)

new_df['potential_parents'] = np.nan

new_df[new_df['intersect_keywords'] == {}].loc['potential_parents'] = new_df[new_df['intersect_keywords'] == {}]['all_keywords'].apply(lambda x: list(set(x)))

def fun(x):
  if(x['intersect_keywords'] == {}):
    # print(x['all_keywords'][0])
    x.loc['potential_parents'] = list(set(x['all_keywords']))
  else:
    max_val = max(x['intersect_keywords'].items(), key=lambda x: x[1])[1]

    listOfKeys = []
    for key, value in x['intersect_keywords'].items():
      if value == max_val:
          listOfKeys.append(key)

    x['potential_parents'] = listOfKeys

  return x

def get_avg_similarity_score(keywords, word):
  word_embedding = model.encode([word])

  score = 0
  for k in keywords:
    keyword_embedding = model.encode([k])
    score += cosine_similarity(word_embedding,keyword_embedding)[0][0]
  
  return score / len(keywords)

def Final_Parent(x):
  max_key = ''
  max_val = 0
  print(x['gp_id'])
  for parent in x['potential_parents']:
    val = get_avg_similarity_score(x['all_keywords'],parent)
    if(val > max_val):
      max_val = val
      max_key = parent
    print(val,max_val,max_key)
    
  x['parent'] = max_key
  print(x)
  return x

new_df['parent'] = ''
  
for i in new_df.index:
  new_df.loc[i] = fun(new_df.loc[i])
  new_df.loc[i] = Final_Parent(new_df.loc[i])

new_df
# new_df['intersect_keywords'].apply(lambda x: max(x.items(), key=lambda x: x[1]))
new_df.to_csv('output.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

42
0.8266095519065857 0.8266095519065857 egyptian
gp_id                                                                42
intersect_keywords    {'ancient egypt': 2, 'egyptian': 4, 'egypt': 2...
all_keywords          [egyptian government, egyptian pharaoh, ancien...
potential_parents                                            [egyptian]
parent                                                         egyptian
Name: 0, dtype: object
54
0.5005262142845562 0.5005262142845562 seaweed
0.46273442783526014 0.5005262142845562 seaweed
gp_id                                                                54
intersect_keywords    {'seaweed': 5, 'seaweed seaweed': 5, 'seaweed ...
all_keywords          [kelp, brown seaweed, red seaweed, green seawe...
potential_parents                            [seaweed, seaweed seaweed]
parent                                                          seaweed
Name: 1, dtype: object
214
0.5988547563552856 0.5988547563552856 atom electron
0.6318554580211639 0.63185545802

In [ ]:
i = 2
print(new_df['intersect_keywords'].apply(lambda top_keywords: sorted(top_keywords, key=lambda k: list(k.values())[0],reverse = True))[i])
total_df[total_df['group_id'] == new_df['gp_id'][i]]


AttributeError: ignored

# GPT-2 Template Generation


In [ ]:
# You'll have to restart your module at this point

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)



In [ ]:
# import pandas as pd
# pd.read_csv('output.csv')

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

In [ ]:
!pip3 install gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
%cd gdrive/MyDrive/Narrative\ Arc\ PE

In [ ]:
model_name = '345M'
run_name = 'run_wiki_oneline_definition_new_re_2'

In [ ]:
#tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

In [ ]:
import random
def ChooseTemplate(keywords,parent_keyword,parent_keyword_def=None):
  if(parent_keyword_def == None):
    raise ValueError("Please Specify the Definition")

  templates = [
    "In this section, you will learn about {parent_keyword}. {definition}. Specifically, you will learn about " + '{}, ' * (len(keywords) - 1) + '{} ' + ".",
    "In this segment, you would be learning about the {parent_keyword}. {definition}. This segment has {len(keywords)} learning resources. The different concepts covered in this segment are " + '{}. ' * (len(keywords) - 1) + '{} ',
    "This section discusses about " + '{}, ' * (len(keywords) - 1) + '{} ' + " which is about {parent_keyword}. {definition}. The different learning resources will discuss " + '{}, ' * (len(keywords) - 1) + '{} ' + ". Lets get started!",
    "The section is about the {parent_keyword}. {definition}. In the first learning resource," +  ", you would learn about {}, then " * (len(keywords) - 1) + " and finally, you will learn about {}.",
    "The section is about the {parent_keyword}. {definition}. In the first learning resource," +  ", you would learn about {}, then " * (len(keywords) - 1) + " and finally, you will learn about {}. Lets get started!",
    "When you hear the terms " + '{}, ' * (len(keywords) - 1) + '{} ' + ", what comes to your mind??" + "\nYes, In the next section we discuss about {parent_keyword}. {definition}. Lets get started!"
  ]
  
  template = random.choice(templates)
  
  return template.format(parent_keyword = parent_keyword,definition = parent_keyword_def,*keywords)

def get_generated_output(keywords, parent_keyword):
  keywords = list(set(keywords))
  text = ", ".join(keywords)
  text = str(text)
  parent_keyword = str(parent_keyword)
  text = "<|startoftext|> " +  parent_keyword + " # "
  out = gpt2.generate(sess,
              run_name = run_name,
              checkpoint_dir='checkpoint',
              length=40,
              return_as_list=True,
              temperature=0.4,
              prefix=text,
              truncate = "<|endoftext|>",
              include_prefix = True, 
              nsamples=1,
              batch_size=1
              )
  out = out[0]
  print(out)
  # print(out.split('#')[-1])
  
  # raise ValueError("Wait for some time")

  return ChooseTemplate(keywords,parent_keyword,out.split('#')[-1])

In [ ]:

# out.split('#')[0].split()[-1].strip()

In [ ]:
import ast
import pandas as pd
# Read your output file here to generate the other portions
new_df = pd.read_csv('/content/output.csv')

Overview = []
new_df
for i in new_df.index:
  keywords= ast.literal_eval(new_df['all_keywords'].loc[i])
  parent_keyword = new_df['parent'].loc[i]
  print(keywords,parent_keyword)
  Overview.append(get_generated_output(list(set(keywords)), parent_keyword))
  print(Overview)

# Overview
new_df['overview'] = Overview
new_df.to_csv('output.csv')


In [ ]:

test_Overview = ['This section discusses about wavelength, wave amplitude, practice wavelength and amplitude, wave frequency which is about waves.']
# new_df['overview'] = Overview
# new_df.to_csv('output.csv')
# new_df['overview'] = Overview
# new_df.to_csv('output_final.csv')
test_Overview

['This section discusses about wavelength, wave amplitude, practice wavelength and amplitude, wave frequency which is about waves.']

In [ ]:
import pandas as pd
df = pd.read_csv("Data/New_Dataset_With_Better_Keywords/New_Dataset_Keywords_Single.csv")
df["BERT_Keywords"] = df["BERT_Keywords"].apply(eval)
df.head(2)

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/Narrative Arc PE/Intersection_within_regions_final.csv')
data.head(2)

In [ ]:
# The summary is the first choice before description from the existig data.
# This function checks if summary is usable or not.
import numpy as np
def removing_some_words(sample_str):
    if "google drive sign" in sample_str.lower():
        return False
    elif "page youre looking" in sample_str.lower():
        return False
    elif "page you're looking" in sample_str.lower():
        return False
    elif "oops" in sample_str.lower():
        return False
    elif "something went wrong" in sample_str.lower():
        return False
    elif "went wrong" in sample_str.lower():
        return False
    
    return True
    
  
def updated_summaries(group_id):
  temp_df = df[df["group_id"] == group_id]
  tempdf_summaries = []
  for ii, row in temp_df.iterrows():
    if removing_some_words(str(row["Summarization"])):
      tempdf_summaries.append(str(row["Summarization"]))
     
    else:
      tempdf_summaries.append(str(row["description"]))

  return tempdf_summaries

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [ ]:
# Scoring for definiton model
def getScore(group_id):

  summaries = updated_summaries(group_id)
  summary = list(data[data["group_id"] == group_id]["Overview"].values)
  summaries.extend(summary)
  summary = None
  summaries = [summ for summ in summaries if summ != np.nan]

  try:
    features = embed(summaries)
    summaries = None
    corr = np.inner(features, features)
    # print(corr)
    return np.nanmean(corr[-1][:-1])
  except Exception as e:
    print(e)
    return np.nan

In [ ]:
data["GPT2_Template_score"] = data.apply(lambda row: getScore(row["group_id"]), axis = 1)
# data.to_csv('/content/gdrive/MyDrive/Narrative Arc PE/Intersection_within_regions_final.csv', index=False)

In [ ]:
np.nanmean(np.array(data["GPT2_Template_score"].values))

0.36604677794086193